# Memory Update with Mistral 7B Instruct Notebook

This notebook demonstrates and tests **memory update operations** using the Mistral 7B Instruct model. It provides a comprehensive testing environment for intelligent memory management and updates.

## **Core Functionality**
- **Memory Update Engine**: Intelligent system for updating existing memories based on new facts
- **Memory Message Builder**: Creates structured prompts for comparing old memories with new facts
- **XML Result Parser**: Robust parsing of AI-generated memory update operations
- **Memory Validation**: Comprehensive validation of memory update operations

## **Memory Operations**
The system supports four key memory operations:
- **ADD**: Create new memories for previously unknown facts
- **UPDATE**: Modify existing memories when facts change or are refined
- **DELETE**: Remove memories when facts are explicitly negated
- **NONE**: Keep memories unchanged when they remain valid

## **Model Integration**
- **Local Mistral 7B**: Uses Ollama's OpenAI-compatible API for local inference
- **Structured Output**: Generates XML-formatted memory update instructions
- **Memory Consolidation**: Intelligently merges related memories

## **Advanced Features**
- **Memory Validation**: Ensures XML structure and content validity
- **Conflict Resolution**: Handles contradictions and preference changes
- **Memory Tracking**: Maintains history of memory changes
- **JSON Conversion**: Converts parsed results to downstream formats

## **Use Cases**
- Testing memory update accuracy with different fact scenarios
- Validating memory consolidation and conflict resolution
- Debugging memory update workflows and XML parsing
- Evaluating AI model performance on memory management tasks

This notebook serves as a comprehensive testing suite for the memory update and management capabilities, ensuring that user memories stay accurate and up-to-date over time.


In [34]:
import json
import re
from xml.etree import ElementTree as ET

def build_memory_messages(old_memory, retrieved_facts):
    system = '''
You are a memory manager for a system.
You must compare a list of **retrieved facts** with the **existing memory** (an array of `{id, text}` objects).  
For each memory item, decide one of four operations: **ADD**, **UPDATE**, **DELETE**, or **NONE**.  
Your output must follow the exact XML format described.

---

## Rules
1. **ADD**:  
   - If a retrieved fact is new (no existing memory on that topic), create a new `<item>` with a new `id` (numeric, non-colliding).
   - Always include `<text>` with the new fact.

2. **UPDATE**:  
   - If a retrieved fact replaces, contradicts, or refines an existing memory, update that memory instead of deleting and adding.  
   - Keep the same `id`.  
   - Always include `<text>` with the new fact.  
   - Always include `<old_memory>` with the previous memory text.  
   - If multiple memories are about the same topic, update **all of them** to the new fact (consolidation).

3. **DELETE**:  
   - Use only when a retrieved fact explicitly invalidates or negates a memory (e.g., “I no longer like pizza”).  
   - Keep the same `id`.  
   - Always include `<text>` with the old memory value so the XML remains well-formed.

4. **NONE**:  
   - If the memory is unchanged and still valid.  
   - Keep the same `id`.  
   - Always include `<text>` with the existing value.

---

## Output format (strict XML only)

<result>
  <memory>
    <item id="STRING" event="ADD|UPDATE|DELETE|NONE">
      <text>FINAL OR EXISTING MEMORY TEXT HERE</text>
      <!-- Only for UPDATE -->
      <old_memory>PREVIOUS MEMORY TEXT HERE</old_memory>
    </item>
  </memory>
</result>

---

## Examples

### Example 1 (Preference Update)
Old: `[{"id": "0", "text": "My name is John"}, {"id": "1", "text": "My favorite fruit is oranges"}]`  
Facts: `["My favorite fruit is apple"]`  

Output:
<result>
  <memory>
    <item id="0" event="NONE">
      <text>My name is John</text>
    </item>
    <item id="1" event="UPDATE">
      <text>My favorite fruit is apple</text>
      <old_memory>My favorite fruit is oranges</old_memory>
    </item>
  </memory>
</result>

### Example 2 (Contradiction / Deletion)
Old: `[{"id": "0", "text": "I like pizza"}]`  
Facts: `["I no longer like pizza"]`  

Output:
<result>
  <memory>
    <item id="0" event="DELETE">
      <text>I like pizza</text>
    </item>
  </memory>
</result>

---

**Important constraints**:
- Never output both DELETE and ADD for the same topic; use UPDATE instead.  
- Every `<item>` must contain `<text>`.  
- Only include `<old_memory>` for UPDATE events.  
- Do not output any text outside `<result>...</result>`.

'''
    prompt = (
        "Old: " + json.dumps(old_memory, ensure_ascii=False) + "\n" +
        "Facts: " + json.dumps(retrieved_facts, ensure_ascii=False) + "\n" +
        "Output:"
    )

    return [
        {"role": "system", "content": system.strip()},
        {"role": "user", "content": prompt}
    ]

def parse_memory_xml(xml_str: str) -> List[MemoryItem]:
    """
    Parse and validate the memory XML.

    Changes from your original:
    - UPDATE items no longer *require* <old_memory>. If missing, old_memory=None.
    - <old_memory> is still forbidden for non-UPDATE events.
    """
    # First extract XML if it's embedded in other content
    xml_str = extract_xml_from_content(xml_str)

    # Clean and validate
    xml_str = clean_and_validate_xml(xml_str)

    try:
        root = ET.fromstring(xml_str.strip())
    except ET.ParseError as e:
        print(f"\nXML Parse Error: {e}")
        print("This usually means:")
        print("- Unclosed tags (e.g., <item> without </item>)")
        print("- Mismatched tags (e.g., <item> closed with </memory>)")
        print("- Invalid characters in XML")
        print("- Missing quotes around attribute values")
        raise MemoryXMLParseError(f"Invalid XML: {e}") from e

    if root.tag != "result":
        raise MemoryXMLParseError("Root element must be <result>.")

    memory = root.find("memory")
    if memory is None:
        raise MemoryXMLParseError("<memory> section is required.")

    items: List[MemoryItem] = []
    seen_ids = set()

    for item in memory.findall("item"):
        # Attributes
        item_id = item.get("id")
        event = item.get("event")

        if not item_id:
            raise MemoryXMLParseError("<item> is missing required 'id' attribute.")
        if not NUMERIC_ID.match(item_id):
            raise MemoryXMLParseError(f"id must be numeric: {item_id!r}")
        if item_id in seen_ids:
            raise MemoryXMLParseError(f"Duplicate id detected: {item_id}")
        seen_ids.add(item_id)

        if event not in ALLOWED_EVENTS:
            raise MemoryXMLParseError(f"Invalid event {event!r} for id {item_id}.")

        # Children
        text_el = item.find("text")
        if text_el is None or (text_el.text or "").strip() == "":
            raise MemoryXMLParseError(f"<text> is required and non-empty for id {item_id}.")
        text_val = (text_el.text or "").strip()

        old_el = item.find("old_memory")
        old_val = (old_el.text or "").strip() if old_el is not None else None

        # Event-specific validation
        if event == "UPDATE":
            # ALLOW missing/empty <old_memory>; just keep None if not present
            pass
        else:
            # For non-UPDATE, <old_memory> must not appear
            if old_el is not None:
                raise MemoryXMLParseError(f"<old_memory> must only appear for UPDATE (id {item_id}).")

        items.append(MemoryItem(id=item_id, event=event, text=text_val, old_memory=old_val))

    if not items:
        raise MemoryXMLParseError("No <item> elements found in <memory>.")

    return items


def items_to_json(items: List[MemoryItem]) -> Dict[str, Any]:
    """Convert parsed items to JSON; only include old_memory when present."""
    out: List[Dict[str, Any]] = []
    for it in items:
        obj: Dict[str, Any] = {"id": it.id, "event": it.event, "text": it.text}
        if it.event == "UPDATE" and it.old_memory:  # include only if non-empty
            obj["old_memory"] = it.old_memory
        out.append(obj)
    return {"memory": out}

old_memory = [
    {"id": "0", "text": "My name is John"},
    {"id": "1", "text": "My favorite fruit is orange"}
]
# retrieved_facts = ["My favorite fruit is apple"]
retrieved_facts = [
    "User is aware of the potential dangers of advanced AI",
    # "User mentions concern about AGI being able to hide problems and manipulate humans",
    "User notes that standard safety methods and societal processes may not be sufficient for dealing with AGI",
    "User provides examples of other technologies (CFCs, planes, nuclear power) and their issues that were later addressed",
    "User mentions the risk of AGI being unrecoverable if control is lost",
    "User states humanity has managed to avoid global thermonuclear war due to development of new technology",
    "User recommends aisafety.info for learning more about AI safety"
  ]

messages = build_memory_messages(old_memory, retrieved_facts)

In [27]:
# from huggingface_hub import snapshot_download
# from pathlib import Path
# from transformers import pipeline

# mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
# mistral_models_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
# chatbot = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
# response = chatbot(messages)




In [28]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

resp = client.chat.completions.create(
    model="mistral:7b-instruct-v0.3-q8_0",
    messages=messages,
    metadata={"langfuse_tags":["local","ollama"]},
)
print(resp.choices[0].message.content)

 <result>
  <memory>
    <item id="0" event="UPDATE">
      <text>My name is John</text>
    </item>
    <item id="1" event="NONE">
      <text>My favorite fruit is orange</text>
    </item>
    <item id="2" event="ADD">
      <text>User is aware of the potential dangers of advanced AI</text>
    </item>
    <item id="3" event="ADD">
      <text>User notes that standard safety methods and societal processes may not be sufficient for dealing with AGI</text>
    </item>
    <item id="4" event="ADD">
      <text>User provides examples of other technologies (CFCs, planes, nuclear power) and their issues that were later addressed</text>
    </item>
    <item id="5" event="ADD">
      <text>User mentions the risk of AGI being unrecoverable if control is lost</text>
    </item>
    <item id="6" event="ADD">
      <text>User states humanity has managed to avoid global thermonuclear war due to development of new technology</text>
    </item>
    <item id="7" event="ADD">
      <text>User recomm

In [35]:
from openai import OpenAI
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

from dataclasses import dataclass, asdict
from typing import List, Optional, Literal, Dict, Any, Union
import xml.etree.ElementTree as ET
import re

Event = Literal["ADD", "UPDATE", "DELETE", "NONE"]

@dataclass(frozen=True)
class MemoryItem:
    id: str
    event: Event
    text: str
    old_memory: Optional[str] = None

class MemoryXMLParseError(ValueError):
    pass

NUMERIC_ID = re.compile(r"^\d+$")
ALLOWED_EVENTS = {"ADD", "UPDATE", "DELETE", "NONE"}

def extract_xml_from_content(content: str) -> str:
    """
    Extract XML from content that might contain other text.
    Looks for content between <result> and </result> tags.
    """
    # Try to find XML block within the content
    import re
    
    # Look for <result>...</result> block
    xml_match = re.search(r'<result>.*?</result>', content, re.DOTALL)
    if xml_match:
        return xml_match.group(0)
    
    # If no <result> tags found, return the original content
    return content

def clean_and_validate_xml(xml_str: str) -> str:
    """
    Clean common XML issues and validate structure.
    """
    xml_str = xml_str.strip()
    
    # Print raw XML for debugging
    print("Raw XML content:")
    print("=" * 50)
    print(repr(xml_str))
    print("=" * 50)
    print("Formatted XML content:")
    lines = xml_str.split('\n')
    for i, line in enumerate(lines, 1):
        print(f"{i:2d}: {line}")
    print("=" * 50)
    
    return xml_str

def extract_assistant_xml_from_openai_response(response) -> str:
    """
    Extract XML content from OpenAI ChatCompletion response.
    Works with both OpenAI API and Ollama via OpenAI-compatible endpoint.
    """
    try:
        # OpenAI ChatCompletion object structure
        return response.choices[0].message.content
    except (AttributeError, IndexError, KeyError) as e:
        raise MemoryXMLParseError(f"Could not extract assistant XML from OpenAI response: {e}") from e


In [36]:
xml = extract_assistant_xml_from_openai_response(resp)
items = parse_memory_xml(xml)
# print(items)                 # list of MemoryItem
print(items_to_json(items))  # {'memory': [...]} for downstream use


Raw XML content:
'<result>\n  <memory>\n    <item id="0" event="UPDATE">\n      <text>My name is John</text>\n    </item>\n    <item id="1" event="NONE">\n      <text>My favorite fruit is orange</text>\n    </item>\n    <item id="2" event="ADD">\n      <text>User is aware of the potential dangers of advanced AI</text>\n    </item>\n    <item id="3" event="ADD">\n      <text>User notes that standard safety methods and societal processes may not be sufficient for dealing with AGI</text>\n    </item>\n    <item id="4" event="ADD">\n      <text>User provides examples of other technologies (CFCs, planes, nuclear power) and their issues that were later addressed</text>\n    </item>\n    <item id="5" event="ADD">\n      <text>User mentions the risk of AGI being unrecoverable if control is lost</text>\n    </item>\n    <item id="6" event="ADD">\n      <text>User states humanity has managed to avoid global thermonuclear war due to development of new technology</text>\n    </item>\n    <item id=

[{'id': '0', 'event': 'UPDATE', 'text': 'My name is John', 'old_memory': None},
 {'id': '1', 'event': 'NONE', 'text': 'My favorite fruit is orange'},
 {'id': '2',
  'event': 'ADD',
  'text': 'User is aware of the potential dangers of advanced AI'},
 {'id': '3',
  'event': 'ADD',
  'text': 'User notes that standard safety methods and societal processes may not be sufficient for dealing with AGI'},
 {'id': '4',
  'event': 'ADD',
  'text': 'User provides examples of other technologies (CFCs, planes, nuclear power) and their issues that were later addressed'},
 {'id': '5',
  'event': 'ADD',
  'text': 'User mentions the risk of AGI being unrecoverable if control is lost'},
 {'id': '6',
  'event': 'ADD',
  'text': 'User states humanity has managed to avoid global thermonuclear war due to development of new technology'},
 {'id': '7',
  'event': 'ADD',
  'text': 'User recommends aisafety.info for learning more about AI safety'}]